In [54]:
import requests
import json
import sentencepiece as SentencePiece
from transformers import MarianMTModel, MarianTokenizer
import re
import os

In [55]:
filename, translate = ['results_comments_phi4:14b-q8_0_czprompt.json', True]
#filename, translate = ['results_comments_phi4.json', True]
#filename, translate = ['results_comments_qwen2.5_32b_inst_q4.json', True]
#filename, translate = ['results_FCGPT_phi4:14b-q8_0_czprompt.json', False]
#filename, translate = ['results_FCGPT_phi4.json', True]

data = []
with open(filename) as f:
    data = json.load(f)
data[0]

{'source': '... pokud chci mit data v bezpeci, tak jedine je mit sifrovana a na Linuxu.',
 'claims': [{'id': 0,
   'claim': 'Chcete-li mít data v bezpečí, musí být zašifrovaná.',
   'checkworthy': True,
   'checkworthy_reason': 'Yes (Tento výrok obsahuje faktickou informaci o tom, že šifrování je nástrojem pro zabezpečení dat, což lze ověřit na základě technologických principů a praxe.)',
   'origin_text': '... pokud chci mit data v bezpeci, tak jedine je mit sifrovana',
   'start': 0,
   'end': 62},
  {'id': 1,
   'claim': 'Linux je způsob, jak udržet data v bezpečí.',
   'checkworthy': True,
   'checkworthy_reason': 'No (I když Linux může být součástí strategií pro zabezpečení dat, tento výrok je příliš obecný a neodkazuje na specifické faktické prvky činící Linux jedinečně bezpečnou.)',
   'origin_text': 'a na Linuxu.',
   'start': 63,
   'end': 75}]}

In [56]:
# Load the model and tokenizer for English to Czech
model_name = "Helsinki-NLP/opus-mt-en-cs"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

In [57]:
def translate_text(input):
    inputs = tokenizer(input, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

for item in data:
    try:
        for claim in item['claims']:
            # Extract text between first '(' and last ')'
            if '(' in claim['checkworthy_reason'] and ')' in claim['checkworthy_reason']:
                start_idx = claim['checkworthy_reason'].find('(') + 1
                end_idx = claim['checkworthy_reason'].rfind(')')
                to_translate = claim['checkworthy_reason'][start_idx:end_idx]
            else:
                to_translate = claim['checkworthy_reason']
            claim['checkworthy'] = claim['checkworthy_reason'].startswith('Yes')
            translated_text = translate_text(to_translate) if translate else to_translate
            claim['checkworthy_reason'] = translated_text
    except:
        print(item)
        # Filter out items that caused errors
        data = [item for item in data if isinstance(item, dict) and 'claims' in item and isinstance(item['claims'], list)]

{'source': 'A peníze na ty dotace si tisknou? Dotace na elektromobilitu fungují jen dokud je elektromobilita minoritní záležitostí.', 'claims': {'Peníze na dotace se tisknou.': {'text': 'A peníze na ty dotace si tisknou?', 'start': 0, 'end': 33}, 'Dotace na elektromobilitu fungují, dokud je elektromobilita minoritní záležitostí.': {'text': 'Dotace na elektromobilitu fungují jen dokud je elektromobilita minoritní záležitostí.', 'start': 34, 'end': 119}}}
{'source': 'Ano, to samozřejmě lze. Ale pokud by na silnicích zůstala převážně osobní doprava, tak údržba bude stát 5-10% toho, co dnes.', 'claims': {'Údržba by stála 5-10% toho, co dnes, pokud na silnicích zůstala převážně osobní doprava.': {'text': 'Ano, to samozřejmě lze. Ale pokud by na silnicích zůstala převážně osobní doprava, tak údržba bude stát 5-10% toho, co dnes.', 'start': 0, 'end': 124}}}
{'source': 'Já si marně vzpomínám, kdy politici místo restrikcí něco povolili. Snad prodloužení intervalu technické u motorek. Jinak se p

In [58]:
output_filename = os.path.join('./filtered', filename)
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)